In [57]:
from textblob import TextBlob
import math
from collections import Counter, defaultdict
import tweepy as tw
import json
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
%matplotlib inline
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [58]:
api_key = "HAcIeikl6eRheQP1oKYGDwPx0"
api_secret = "gEWmqG7QVXzudnviUXDo98L2UKGYg9PNqy1bAR1geClkHjqBGk"
access_token = "1481095074465579008-1pApYlU4HRJWpbAXcbdVEVGzdACuEz"
access_token_secret = "htgD81H4mOxXw5MglAljFRHhcfGAuwsNomXVqvGw0Obwe"
auth = tw.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth,wait_on_rate_limit=True)

In [59]:
number_of_tweets = 1000
tweets = []
user_id =[]
created_at = []
keyword_list = ['News', 'Stock'] # , 'Customers', 'Employees', 'Company', 'Price', 'Shareholders', 'Market', 'Soaring', 'Trends'
company = 'Amazon'
final_kw_list = []

# Crawl data from keywords
for i in range (len(keyword_list)):
    kw = str(company + ' ' + keyword_list[i])
    print(kw)
    final_kw_list.append(kw)
    for i in tw.Cursor(api.search, q = kw , lang ="en", since = "2022-01-14",tweet_mode = "extended").items(number_of_tweets):
        tweets.append(i.full_text)
        user_id.append(i.user.id)
        created_at.append(i.created_at)

Amazon News
Amazon Stock


In [60]:
df = pd.DataFrame({"Created_at": created_at,"User_id": user_id,"Tweets":tweets})
df

,Created_at,User_id,Tweets
0,2022-02-07 08:44:18,4419993470,Stock Market Today: Amazon's Historic Day Push...
1,2022-02-07 08:43:49,1299877337148936192,WOW: Priyanka Chopra Jonas to be seen in Holly...
2,2022-02-07 08:42:50,15482079,Net sales at Amazon were up by 9% year on year...
3,2022-02-07 08:42:50,1267197215635726336,&lt;Ethereum Price&gt;\nETH/USDT(Binance): 306...
4,2022-02-07 08:41:32,1483083938151428096,@UNITY_WINS @DougJBalloon Do you like #Amazon ...
...,...,...,...
1995,2022-02-06 19:21:11,1049736336058408960,They are still in stock on Amazon.\n\nGet your...
1996,2022-02-06 19:21:09,1290791241349763072,RT @ColinBurnett16: I'm trying to give my book...
1997,2022-02-06 19:20:39,499672969,(US)In Stock: MSI 6600 XT MECH 2X OC - 599.99\...
1998,2022-02-06 19:20:00,1377519267143376898,PowerColor Red Devil AMD Radeon™ RX 6900 XT Ga...


In [61]:
df = df[~df.Tweets.str.contains("RT")]
df = df.reset_index(drop = True)

In [62]:
def clean_up_tweet(text):
    text = re.sub(r'@[A-Za-z0-9_]+','',text) # Remove mentions @
    text = re.sub(r'#', '', text) # Remove hastag symbol
    text = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', text) # Remove hyper link
    text = re.sub('[^A-Za-z0-9]+', ' ', text) # Remove special characters
    text = re.sub(r'[0-9]+', '', text)
    #Remove stop words
    res = ''
    for word in text.split():
        if word not in stopwords.words('english'):
            res += WordNetLemmatizer().lemmatize(word) + ' '

    return res

In [63]:
df['Tweets'] = df['Tweets'].apply(clean_up_tweet)
df = df.drop_duplicates(subset='Tweets')
df = df.reset_index(drop=True)
df

,Created_at,User_id,Tweets
0,2022-02-07 08:44:18,4419993470,Stock Market Today Amazon Historic Day Pushes ...
1,2022-02-07 08:43:49,1299877337148936192,WOW Priyanka Chopra Jonas seen Hollywood actio...
2,2022-02-07 08:42:50,15482079,Net sale Amazon year year bn bn three month De...
3,2022-02-07 08:42:50,1267197215635726336,lt Ethereum Price gt ETH USDT Binance Check Pr...
4,2022-02-07 08:41:32,1483083938151428096,Do like Amazon Gift Card Great news This Amazo...
...,...,...,...
640,2022-02-06 19:24:33,1355424061027876867,Stocks climb investor weigh surprising job rep...
641,2022-02-06 19:23:30,138925377,The Motorola MA Wireless back stock Amazon
642,2022-02-06 19:22:49,1306463816226660352,Amazon antonline Store Intel Core KF Hexadeca ...
643,2022-02-06 19:21:27,16557144,Amazon must stock tank surely


In [64]:
# Create function to get subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

# Create function to get polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

# Function compute negative, positive
def getAnalysis(score):
    if score < 0:
        return np.int_(0)
    elif score > 0:
        return np.int_(1)
    else:
        return 'Neutral'

df['Subjectivity'] = df['Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Tweets'].apply(getPolarity)
df['Label'] = df['Polarity'].apply(getAnalysis)
df.drop(df.loc[df['Label']=='Neutral'].index, inplace=True)
df = df.reset_index(drop=True)
df

,Created_at,User_id,Tweets,Subjectivity,Polarity,Label
0,2022-02-07 08:44:18,4419993470,Stock Market Today Amazon Historic Day Pushes ...,0.250000,0.125000,1
1,2022-02-07 08:43:49,1299877337148936192,WOW Priyanka Chopra Jonas seen Hollywood actio...,0.366667,0.066667,1
2,2022-02-07 08:41:32,1483083938151428096,Do like Amazon Gift Card Great news This Amazo...,0.583333,0.566667,1
3,2022-02-07 08:37:31,1352234446402048003,Businesshala News Exclusive Amazon Other Poten...,0.687500,-0.062500,0
4,2022-02-07 08:36:34,1156926992119488513,S amp P rise end best week year far Amazon pus...,0.600000,0.450000,1
...,...,...,...,...,...,...
369,2022-02-06 19:29:50,1027263632613298177,A week record A day Facebook owner Meta Platfo...,1.000000,0.500000,1
370,2022-02-06 19:27:43,1449083262408740866,A day Facebook owner Meta Platforms suffered d...,1.000000,1.000000,1
371,2022-02-06 19:27:26,34146256,Think investing method world best lifetime sid...,0.446667,0.324167,1
372,2022-02-06 19:24:33,1355424061027876867,Stocks climb investor weigh surprising job rep...,0.477273,0.418182,1


In [65]:
class Tokenizer:

    def clean(self, text):
        no_html = BeautifulSoup(text).get_text()
        clean = re.sub("[^a-z\s]+", " ", no_html, flags=re.IGNORECASE)
        return re.sub("(\s+)", " ", clean)


    def tokenize(self, text):
        clean = self.clean(text).lower()
        stopwords_en = stopwords.words("english")
        return [w for w in re.split("\W+", clean) if not w in stopwords_en]

In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(min_df = 5, max_df = 0.8, sublinear_tf = True, use_idf = True)

tfidf = tfidf_vectorizer.fit_transform(df['Tweets'].values)

In [67]:
from sklearn.model_selection import train_test_split

X = df['Tweets'].values
y = df['Label'].values.astype('float')

X_train, X_test, y_train, y_test = train_test_split(tfidf, y, test_size=0.2, random_state=42)

In [68]:
from sklearn import svm
from sklearn.metrics import classification_report
classifier_linear = svm.SVC(kernel='linear')
classifier_linear.fit(X_train, y_train)
prediction_linear = classifier_linear.predict(X_test)

print(classification_report(y_test, prediction_linear))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        24
         1.0       0.68      1.00      0.81        51

    accuracy                           0.68        75
   macro avg       0.34      0.50      0.40        75
weighted avg       0.46      0.68      0.55        75



C:\Users\hlinh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hlinh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hlinh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Testing

In [69]:
review = """SUPERB, I AM IN LOVE IN THIS PHONE"""
review_vector = tfidf_vectorizer.transform([review])
print(classifier_linear.predict(review_vector))

[1.]


In [70]:
review = """Do not purchase this product. My cell phone blast when I switched the charger"""
review_vector = tfidf_vectorizer.transform([review])
print(classifier_linear.predict(review_vector))

[1.]


In [71]:
review = """I received defective piece display is not working properly"""
review_vector = tfidf_vectorizer.transform([review])
print(classifier_linear.predict(review_vector))

[0.]
